# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [3]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

In [8]:
cur.execute("""
                SELECT name from sqlite_master
                WHERE type='table'
                """)
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df.head()

,name
0,productlines
1,offices
2,customers
3,employees
4,orderdetails


In [9]:
cur.execute("""
                SELECT * from employees;
                """)
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df.head()

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)


In [10]:
cur.execute("""
                SELECT * from offices;
                """)
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df.head()

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan


## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [20]:
cur.execute("""
                SELECT lastname,firstname,city from employees
                JOIN offices
                USING(officeCode)
                WHERE city == 'Boston'
             """)

df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df

,lastName,firstName,city
0,Firrelli,Julie,Boston
1,Patterson,Steve,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [43]:
cur.execute("""SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS n_employees
               FROM offices AS o 
               LEFT JOIN employees AS e
               USING(officeCode)
               GROUP BY officeCode
               HAVING n_employees == 0;
            """)


df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df

,officeCode,city,n_employees
0,27,Boston,0


## Write 3 Questions of your own and answer them

In [45]:
#Which How many and What are the names of the people in the Tokyo Office

cur.execute("""SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS n_employees
               FROM offices AS o 
               LEFT JOIN employees AS e
               USING(officeCode)
               WHERE officeCode == 5
               GROUP BY officeCode;
            """)


df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df

,officeCode,city,n_employees
0,5,Tokyo,2


In [48]:
#What are the names of the employees in the tokyo office?

cur.execute("""SELECT lastname,firstname from employees
                WHERE officeCode == 5
            """)


df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df.head()

,lastName,firstName
0,Nishi,Mami
1,Kato,Yoshimi


In [55]:
#What customer has ordered the most product?

cur.execute(""" SELECT COUNT(orderNumber) as num_orders, customerNumber 
                FROM orders
                GROUP BY customerNumber
                ORDER BY num_orders DESC
                LIMIT 1;
            
                
            """)


df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df.head()

,num_orders,customerNumber
0,26,141


## Level Up: Display the names of every individual product that each employee has sold

In [59]:


cur.execute(""" SELECT lastname,firstname, productName 
                FROM employees as e
                JOIN customers as c
                ON employeeNumber = c.salesRepEmployeeNumber
                JOIN orders as o
                USING(customerNumber)
                JOIN orderdetails as od
                USING(orderNumber)
                JOIN products as p
                USING(productCode)
            """)


df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()

OperationalError: cannot join using column productName - column not present in both tables

## Level Up: Display the Number of Products each employee has sold

In [63]:
cur.execute(""" SELECT lastname,firstname, COUNT(productName) 
                FROM employees as e
                JOIN customers as c
                ON employeeNumber = c.salesRepEmployeeNumber
                JOIN orders as o
                USING(customerNumber)
                JOIN orderdetails as od
                USING(orderNumber)
                JOIN products as p
                USING(productCode)
                GROUP BY lastname
                ORDER BY firstname;
            """)


df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()



15


,lastName,firstName,COUNT(productName)
0,Fixter,Andy,185
1,Jones,Barry,220
2,Tseng,Foon Yue,142
3,Vanauf,George,211
4,Hernandez,Gerard,396


In [65]:
## the previous "level up" question and call a pandas .groupby method like so:

cur.execute(""" SELECT lastname,firstname, productName 
                FROM employees as e
                JOIN customers as c
                ON employeeNumber = c.salesRepEmployeeNumber
                JOIN orders as o
                USING(customerNumber)
                JOIN orderdetails as od
                USING(orderNumber)
                JOIN products as p
                USING(productCode)
            """)


df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()

df.groupby(['firstName','lastName']).count()

2996


productName
firstName lastName              
Andy      Fixter             185
Barry     Jones              220
Foon Yue  Tseng              142
George    Vanauf             211
Gerard    Hernandez          396
Julie     Firrelli           124
Larry     Bott               236
Leslie    Jennings           331
          Thompson           114
Loui      Bondur             177
Mami      Nishi              137
Martin    Gerard             114
Pamela    Castillo           272
Peter     Marsh              185
Steve     Patterson          152

## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!